In [ ]:
import numpy as np
import time
from collections import namedtuple
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import qiskit as q
import qiskit.tools.jupyter
from qiskit.pulse import pulse_lib as _  # only want to call pulse_lib as q.pulse.pulse_lib

In [ ]:
import pulse_compiler_helper_fns

In [ ]:
q.IBMQ.load_account() # Load account from disk
q.IBMQ.providers()    # List all available providers

In [ ]:
provider = q.IBMQ.get_provider(group='bes-qis')
backend = provider.get_backend('ibmq_boeblingen')
system = q.pulse.PulseChannelSpec.from_backend(backend)
cmd_def = q.pulse.CmdDef.from_defaults(backend.defaults().cmd_def, backend.defaults().pulse_library)
measure = cmd_def.get('measure', qubits=backend.configuration().meas_map[0])

In [ ]:
schedules = []
for theta in np.linspace(0, np.pi, 13):
    for basis in ['X', 'Y', 'Z']:
        circuit = q.QuantumCircuit(1)
        circuit.rx(theta, 0)
        decomposed_circuit = circuit.decompose()
        pulse_compiler_helper_fns.update_basis_gates_and_cmd_def(decomposed_circuit, backend, system, cmd_def)
        transpiled_circuit = q.transpile(circuit, backend, optimization_level=1)
        schedule = q.schedule(transpiled_circuit, backend=backend, cmd_def=cmd_def)
        if basis == 'Y':  # apply H Sdag
            schedule |= cmd_def.get('u1', qubits=[0], P0=-np.pi/2) << schedule.duration
            schedule |= cmd_def.get('u2', qubits=[0], P0=0, P1=np.pi) << schedule.duration
        elif basis == 'X':  # apply H
            schedule |= cmd_def.get('u2', qubits=[0], P0=0, P1=np.pi) << schedule.duration
        schedule |= measure << schedule.duration
        schedules.append(schedule)

In [ ]:
shots = 200
job = execute(schedules, backend=backend, shots=shots)
print(job.job_id())

In [ ]:
result = job.result()

In [ ]:
marginal_counts = q.ignis.verification.tomography.data.marginal_counts
for i in range(13):
    x_counts = marginal_counts(result.get_counts(3 * i), meas_qubits=[0])
    y_counts = marginal_counts(result.get_counts(3 * i + 1), meas_qubits=[0])
    z_counts = marginal_counts(result.get_counts(3 * i + 2), meas_qubits=[0])
    
    # bloch vector components:
    x = (x_counts.get('0', 0) / shots - 0.5) * 2
    y = (y_counts.get('0', 0) / shots - 0.5) * 2
    z = (z_counts.get('0', 0) / shots - 0.5) * 2
    print(theta[i], x, y, z)